## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Srednekolymsk,67.4500,153.6833,34.97,88,100,11.54,RU,NaN
1,1,Upernavik,72.7868,-56.1549,33.84,67,82,3.94,GL,NaN
2,2,Colares,38.7992,-9.4469,69.48,67,66,7.63,PT,NaN
3,3,Inhambane,-23.8650,35.3833,66.85,72,21,11.68,MZ,NaN
4,4,Matamba,-8.9833,33.9667,50.72,79,8,6.42,TZ,NaN


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
6,6,Alofi,-19.0595,-169.9187,78.69,83,37,16.11,NU,NaN
21,21,Guanica,17.9716,-66.9080,84.06,84,57,1.99,PR,NaN
22,22,Acapulco,16.8634,-99.8901,80.49,84,99,2.68,MX,NaN
23,23,Qidong,31.8128,121.6561,85.03,82,25,5.46,CN,NaN
24,24,Sao Filipe,14.8961,-24.4956,78.91,78,10,5.30,CV,NaN
31,31,Kavaratti,10.5669,72.6420,81.77,77,10,16.26,IN,NaN
33,33,Tautira,-17.7333,-149.1500,82.42,68,85,12.88,PF,NaN
36,36,Kokopo,-4.3520,152.2633,85.30,60,62,7.38,PG,NaN
37,37,Catarman,12.4989,124.6377,81.73,80,100,10.85,PH,NaN
41,41,Dori,14.0354,-0.0345,82.74,64,20,10.16,BF,NaN


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             225
City                   225
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Country                225
Current Description      0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Alofi,NU,78.69,-19.0595,-169.9187,
21,Guanica,PR,84.06,17.9716,-66.9080,
22,Acapulco,MX,80.49,16.8634,-99.8901,
23,Qidong,CN,85.03,31.8128,121.6561,
24,Sao Filipe,CV,78.91,14.8961,-24.4956,
31,Kavaratti,IN,81.77,10.5669,72.6420,
33,Tautira,PF,82.42,-17.7333,-149.1500,
36,Kokopo,PG,85.30,-4.3520,152.2633,
37,Catarman,PH,81.73,12.4989,124.6377,
41,Dori,BF,82.74,14.0354,-0.0345,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    }

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
       
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        pass

# len(hotels["results"])


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Alofi,NU,78.69,-19.0595,-169.9187,Taloa Heights
21,Guanica,PR,84.06,17.9716,-66.9080,Copamarina Beach Resort & Spa
22,Acapulco,MX,80.49,16.8634,-99.8901,HS Hotsson Smart Acapulco
23,Qidong,CN,85.03,31.8128,121.6561,Hilton Garden Inn Qidong
24,Sao Filipe,CV,78.91,14.8961,-24.4956,Hotel Xaguate
...,...,...,...,...,...,...
683,Aquiraz,BR,80.69,-3.9014,-38.3911,Pousada Rua Camurupim
684,Sabha,LY,83.19,27.0377,14.4283,Foundation Taaj Asala for Historical Research ...
685,Hirara,JP,84.47,24.8000,125.2833,Hotel Atoll Emerald Miyakojima
687,Shanghai,CN,84.92,31.2222,121.4581,Swissotel Grand Shanghai


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))